In [1]:
%load_ext autoreload
%autoreload 2



from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_auc_score
# noinspection PyUnresolvedReferences
from hcve_lib.tracking import load_run_results
from hcve_lib.utils import notebook_init, run_parallel
from hcve_lib.evaluation_functions import map_inverse_weight


notebook_init()


from notebooks.deps.binary_predictive_performance import run_roc_analysis, get_pr_analysis
from notebooks.deps.binary_predictive_performance import get_pr_analysis, get_pr_analysis_ci, plot_pr_ci

from mlflow import set_tracking_uri
from notebooks.deps.config import TIME_POINT_PREDICTION
from deps.common import get_data_cached

from sklearn.metrics import roc_curve
from pandas import DataFrame
from plotly.graph_objs import Figure

from deps.constants import RANDOM_STATE
from hcve_lib.evaluation_functions import average_group_scores, merge_standardize_prediction, merge_predictions, \
    compute_metrics_prediction
from hcve_lib.metrics import BootstrappedMetric
from hcve_lib.tracking import load_group_results
from hcve_lib.visualisation import setup_plotly_style
from hcve_lib.functional import t
import numpy as np
from hcve_lib.metrics import statistic_from_bootstrap
from hcve_lib.functional import reject_none
from plotly import express as px
from hcve_lib.utils import transpose_list
from notebooks.deps.binary_predictive_performance import run_pr_analysis_ci

from config import GROUPS_LCO, GROUPS_10_fold
from hcve_lib.metrics import BinaryMetricFromScore
from hcve_lib.data import binarize_event

set_tracking_uri('http://localhost:5000')

<frozen importlib._bootstrap>:228: RuntimeWarning: sklearn.tree._splitter.Splitter size changed, may indicate binary incompatibility. Expected 360 from C header, got 1160 from PyObject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data, metadata, X, y = get_data_cached()


[Memory]11.0s, 0.2min   : Loading get_data...
____________________________________________get_data cache loaded - 0.2s, 0.0min


In [9]:
y_binarized = binarize_event(10*365, y['data'])

/home/sitnarf/projects/hcve_lib/hcve_lib/data.py:247: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [11]:
inverse_weight_cohorts = map_inverse_weight(data['STUDY'].loc[y_binarized.index])

In [14]:
from hcve_lib.functional import dict_subset

GROUPS = dict_subset(['gb', 'coxnet', 'stacking', 'pcp_hf'], GROUPS_LCO)

In [15]:
TIME_POINT_PREDICTION = 10*365
ITERATIONS = 50

In [16]:
from toolz import valmap 

merged_prediction = valmap(
    lambda group_id: merge_predictions(average_group_scores(load_group_results(group_id))),
    GROUPS,
)



KeyboardInterrupt



In [18]:
from hcve_lib.wrapped_sklearn import DFStandardScaler
import dtale

predictions = DFStandardScaler().fit_transform(DataFrame({
    'gb': merged_prediction['gb']['y_score'],
    'coxnet': merged_prediction['coxnet']['y_score'],
    'pcp_hf': merged_prediction['pcp_hf']['y_score'],
    'stacking': merged_prediction['stacking']['y_score'],
}
))
predictions['y'] = y_binarized
predictions.dropna(inplace=True)
predictions['STUDY'] = data['STUDY'].loc[predictions.index]


dtale.show(predictions.sample(len(predictions), weights=inverse_weight_cohorts, replace=True), host='localhost')

/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/views.py:725: FutureWarning:

['STUDY'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/views.py:1929: FutureWarning:

The default value of regex will change from True to False in a future version.

/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/views.py:1929: FutureWarning:

The default value of regex will change from True to False in a future version.



Exception occurred while processing request: min() takes 1 positional argument but 2 were given
Traceback (most recent call last):
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/views.py", line 2708, in get_column_analysis
    return jsonify(**analysis.build())
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/column_analysis.py", line 137, in build
    return_data, code = self.analysis.build(self)
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/column_analysis.py", line 190, in build
    bin_vals = pd.cut(parent.data[parent.selected_col], bins=self.bins)
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/pandas/core/r

/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/views.py:1929: FutureWarning:

The default value of regex will change from True to False in a future version.

2022-09-25 13:29:48,498 - ERROR    - Exception occurred while processing request: min() takes 1 positional argument but 2 were given
Traceback (most recent call last):
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/views.py", line 2708, in get_column_analysis
    return jsonify(**analysis.build())
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/column_analysis.py", line 137, in build
    return_data, code = self.analysis.build(self)
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9

Exception occurred while processing request: min() takes 1 positional argument but 2 were given
Traceback (most recent call last):
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/views.py", line 2708, in get_column_analysis
    return jsonify(**analysis.build())
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/column_analysis.py", line 137, in build
    return_data, code = self.analysis.build(self)
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/column_analysis.py", line 190, in build
    bin_vals = pd.cut(parent.data[parent.selected_col], bins=self.bins)
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/pandas/core/r

...

2022-09-25 14:30:17,988 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2022-09-25 14:30:18,000 - INFO     - Executing shutdown...


Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/sitnarf/.local

2022-09-25 14:30:18,002 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_